# 02 Exploratory Data Analysis

Notebook goal: initial Exploratory Data Analysis to understand features, data cleanliness and basic (inter)correlations.

## 1. Load data

Data processed in previous notebook

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
# Load into pandas
df = pd.read_parquet("../data/original-data.parquet")

## Explore Data types

In [ ]:
df.dtypes

In [ ]:
df.arrival_day_of_week.unique()

In [ ]:
df.arrival_month_name.unique()

## Convert datetimes

In [ ]:
df2 = df.copy()

In [ ]:
df2.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL = pd.to_datetime(
    df2.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
# mainly nan
# df2.DISCHARGE_READY_DATE.sample(10)
df2.EXPECTED_DISCHARGE_DATE = pd.to_datetime(
    df2.EXPECTED_DISCHARGE_DATE, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
df2.FIRST_START_DATE_TIME_WARD_STAY = pd.to_datetime(
    df2.FIRST_START_DATE_TIME_WARD_STAY, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
df2.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL = pd.to_datetime(
    df2.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
df2.dtypes

## Sanity checks

Does the LENGTH_OF_STAY match start/end dates?

In [ ]:
# check data types before conducting maths
df2[
    [
        "DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL",
        "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL",
    ]
].dtypes

In [ ]:
# Discharge is whole day, admission is datetime
df2[
    [
        "DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL",
        "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL",
    ]
].sample(10)

In [ ]:
# calculate derived LoS
# round up to whole days
df2["DER_los"] = (
    df2["DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL"]
    - df2["START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]
).dt.days + 1

In [ ]:
# quick visual inspection - do they match?
df2[["DER_los", "LENGTH_OF_STAY"]].head(10)

In [ ]:
# check that mean difference is ~ 0 days
df2[["DER_los", "LENGTH_OF_STAY"]].diff(axis=1).LENGTH_OF_STAY.mean()

In [ ]:
# drop the derived column
df2.drop(columns="DER_los", inplace=True)

## Data ordering

How is data ordered?

In [ ]:
# not ordered by local patient id
df2.LOCAL_PATIENT_IDENTIFIER.head(10)

# nb. value_counts() shows repeat visits - could this be feature?

In [ ]:
# not ordered by start-date
df2.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL.head(10)

In [ ]:
# not by end-date
df2.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL.head(10)

In [ ]:
# not by cds
df2.cds_unique_identifier.sample(10)

### Order by arrival date

In [ ]:
df3 = df2.sort_values(by="START_DATE_TIME_HOSPITAL_PROVIDER_SPELL").reset_index(
    drop=True
)

In [ ]:
df3.head(10)

### Missing data

In [ ]:
sns.set(rc={"figure.figsize": (15, 8)})
sns.heatmap(df3.isnull(), cbar=False);

In [ ]:
df3.isnull().sum()

In [ ]:
df3.IS_major.unique()

In [ ]:
df3.IS_major.value_counts()

In [ ]:
# Extract columns with 100% empty values
empty_cols = df3.isnull().sum() == df.shape[0]
empty_cols = empty_cols[empty_cols].index.array

In [ ]:
for col in empty_cols:
    print(f"{col} values: {df[col].unique()}")

In [ ]:
df4 = df3.drop(columns=empty_cols)

In [ ]:
df4[df4.stroke_ward_stay.notna()].shape

In [ ]:
df4.stroke_ward_stay.value_counts()

Question:

1. When does the block of missing data end?


In [ ]:
df4.iloc[df4.presenting_complaint.first_valid_index()][
    ["START_DATE_TIME_HOSPITAL_PROVIDER_SPELL", "presenting_complaint"]
]

In [ ]:
df4.iloc[df4.presenting_complaint.first_valid_index() - 1][
    ["START_DATE_TIME_HOSPITAL_PROVIDER_SPELL", "presenting_complaint"]
]

## Correlation plot

In [ ]:
# Pearson correlation by default:
corr = df4.corr()

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

In [ ]:
print(df4.FIRST_WARD_STAY_IDENTIFIER.unique())
df4.FIRST_WARD_STAY_IDENTIFIER.value_counts()

In [ ]:
# Remove empty/single value columns
df5 = df4.drop(
    columns=[
        "FIRST_WARD_STAY_IDENTIFIER",
        "LENGTH_OF_STAY_IN_MINUTES",
        "START_DATE_HOSPITAL_PROVIDER_SPELL",
        "EXPECTED_DISCHARGE_DATE_TIME",
    ]
)

In [ ]:
# How many negative/invalid LoS are there?
df5[df5.LENGTH_OF_STAY < 0].shape

In [ ]:
# Remove invalid LoS
df6 = df5[df5.LENGTH_OF_STAY > -1]

## Check duplicate rows

In [ ]:
df6[df6.duplicated(keep=False)].sort_values(
    by="START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"
).head()

In [ ]:
df7 = df6.drop_duplicates()

In [ ]:
print(df6.shape)
print(df7.shape)

## Check duplicate columns

In [ ]:
df7.FIRST_START_DATE_TIME_WARD_STAY.equals(df7.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL)

In [ ]:
df7[
    ["FIRST_START_DATE_TIME_WARD_STAY", "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]
].sample(100).diff(axis=1)

In [ ]:
df7.loc[154218]

In [ ]:
(
    df7[["FIRST_START_DATE_TIME_WARD_STAY", "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]]
    .diff(axis=1)
    .START_DATE_TIME_HOSPITAL_PROVIDER_SPELL
    > pd.Timedelta(0)
).sum()

In [ ]:
# Drop duplicate columns
df8 = df7.drop(columns="FIRST_START_DATE_TIME_WARD_STAY")

In [ ]:
df8.shape

## Pandas profiling

In [ ]:
pd.__version__
# note bug with version 1.4.1: https://github.com/ydataai/pandas-profiling/issues/911
# use lower version (e.g. 1.3.5)

In [ ]:
# Dataset large and crashing without minimal=True
profile = ProfileReport(df8, title="Pandas Profiling Report", minimal=True)

In [ ]:
profile

In [ ]:
df8.attendance_type.value_counts()

In [ ]:
# Drop invalid/incomplete columns
df9 = df8.drop(
    columns=[
        "FIRST_REGULAR_DAY_OR_NIGHT_ADMISSION_DESCRIPTION",
        "wait",
        "attendance_type",
        "initial_wait",
        "arrival_day_of_week",
        "arrival_month_name",
    ]
)

In [ ]:
sns.set(rc={"figure.figsize": (15, 8)})
sns.heatmap(df9.isnull(), cbar=False);

## Export snapshot data

In [ ]:
df9.to_parquet("../data/02-eda.parquet")

## Export data dictionary fields

Data dictionary will be created in Google Sheets/Excel

In [ ]:
# Export cols/descriptions for Excel/Google Sheets import
df9.dtypes.to_csv("../data/cols.csv")
df9.describe().to_csv("../data/describe.csv")